In [1]:
!pip install --force-reinstall numpy==2.1.0

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.1.0 which is incompatible.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.1.0 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.1.0 which is incompatible.
streamlit 1.32.0 requires numpy<2,>=1.19.3, but you have numpy 2.1.0 which is incompatible.



  Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [2]:
# 사용방법 : Input 폴더 생성후 내부에 APP으로 촬영한 Basis Images들이 들어있는 폴더를 그대로 복사하고 코드를 실행 
# Output : Output 폴더에 HDR 처리된 Basis Image와 Silhouette Image 생성
# Requirement : Rawpy 사용으로 numpy 2.0.0 이상 필요함
import matplotlib as plt
import rawpy
import PIL
import os
import numpy as np
import cv2 as cv
import glob

def getfolderlist(fname):
    path = "./"+fname+"/"
    file_list = os.listdir(path)
    ret = []
    for i in range(len(file_list)):
        if file_list[i] == ".ipynb_checkpoints":
            continue
        if os.path.isdir(path+file_list[i]) == True:
            ret.append(file_list[i])
    return ret

def getfilelist(fname):
    def getkey(str): # yyyymmdd_hhmmss_imagename(Pattern000~###)_#_option.dng
        str = str.split('_')
        num = str[4].split('.')
        return (str[0]+str[1]), int(str[3]), float(num[0]+'.'+num[1]), str[2]+str[3]
    order = {}
    path = "./"+fname+"/"
    file_list = os.listdir(path)
    ret = [{},{}]
    for i, v in enumerate(file_list):
        Num = len(file_list[i])
        if Num > 4 and file_list[i][-4:Num] == ".dng":
            key, num, opt, name = getkey(file_list[i])
            if num == 0:
                ret[0][opt] = [path,file_list[i],opt]
            else:
                if order.get(opt,-1) == -1:
                    order[opt] = 0
                else:
                    order[opt] += 1
                if ret[1].get(name,-1) == -1:
                    ret[1][name] = [[path,file_list[i],opt,order[opt]]]
                else:
                    ret[1][name].append([path,file_list[i],opt,order[opt]])
    return ret

def OpenImage(image):
    try:
        raw = rawpy.imread(image[0]+image[1])
    except:
        return -1    
    RGB = raw.postprocess(no_auto_bright=True)
    return (raw, RGB)

def SavetoTIFF(image,handle): # for Test
    PIL.Image.fromarray(handle[1]).save("./Output/"+image[1].replace(".dng",".tiff"))
    handle[0].close()
    
def SavetoPNG(image,handle):
    PIL.Image.fromarray(handle[1]).save("./Output/"+image[1].replace(".dng",".png"))
    handle[0].close()

def CloseImage(handle):
    handle[0].close()

def IntensityCorrection(Image_handle,Base_handle,t):  # AX+B -> X
    BaseRGB = Base_handle[t][1]
    ImageRGB = Image_handle[1]
    ImageRGB = np.subtract(ImageRGB,BaseRGB,dtype=np.int16)

    # CalcCurve의 결과를 사용함 : [600, 400, 2400, 1000]
    exp_a = [8.434531449998532, 8.473601770815652, 11.258200415483262]
    exp_b = [0.5911675096624116, 0.5845863077436514, 0.5640701602515074]
    lin_a = [1.1588662162251764, 1.1278004461230864, 1.3570659347609255]
    
    CorrectionCoef = np.array([(lin_a[i]*t)/(exp_a[i]*(t**exp_b[i])) for i in range(3)],dtype=np.float32)
    ImageRGB = np.multiply(ImageRGB,CorrectionCoef,dtype=np.float32)
    
    return (Image_handle[0], np.clip(ImageRGB, 0, 255).astype('uint8'))

def makeHDRfilename(str):
    str = str.split('_')
    return str[0]+"_"+str[1]+"_"+str[2]

def AddPNGfile(img, handle):
    def getkey(str): # yyyymmdd_hhmmss_imagename_#_option.png
        str = str.split('_')
        num = str[4].split('.')
        return str[3], float(num[0]+'.'+num[1])
    key, opt = getkey(img[1])
    return [handle[1],img[1],opt,img[3]]
    
def SavetoHDR(L,f,Cam):
    Num = len(L)
    fname = makeHDRfilename(L[0][1])
    print("HDR Imaging ({}) : {:d} Images are Loaded.".format(fname,Num))
    image_list = [cv.cvtColor(v[0], cv.COLOR_BGR2RGB) for v in L]
    exposure_times = np.array([v[2] for v in L], dtype=np.float32)

    # Merge exposures to HDR image
    merge_debevec = cv.createMergeDebevec()
    hdr_debevec = merge_debevec.process(image_list, times=exposure_times.copy())

    # Tonemap HDR image
    tonemap = cv.createTonemap(gamma=1.0)
    res_debevec = tonemap.process(hdr_debevec.copy())

    # Convert datatype to 8-bit and save
    res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
    res_debevec_8bit = cv.undistort(res_debevec_8bit, Cam[0], Cam[1]) # Undistortion (Remove Lens distortion)
    
    res_debevec_8bit_s = cv.resize(res_debevec_8bit, dsize=(456,608), interpolation=cv.INTER_LINEAR) # 2736x3648 -> 456x608
    #res_debevec_8bit_s = cv.GaussianBlur(res_debevec_8bit_s,(0,0),0.5) #Gaussian Filter
    cv.imwrite("./Output/"+f+"/"+str(L[0][3]).zfill(3)+".png", res_debevec_8bit_s)
    
    print("{} saved.".format("./Output/"+f+"/"+str(L[0][3]).zfill(3)+".png"))
    return res_debevec_8bit

def Init_Undistortion():
    criteria = (cv.TERM_CRITERIA_EPS+cv.TERM_CRITERIA_MAX_ITER, 30 ,0.001)
    wc = 8  ## 체스 보드 가로 패턴 개수 - 1
    hc = 6  ## 체스 보드 세로 패턴 개수 - 1
    images = glob.glob('./Base/*.jpg')

    objp = np.zeros((wc * hc, 3), np.float32)
    objp[:, :2] = np.mgrid[0:wc, 0:hc].T.reshape(-1, 2)
    objpoints = []
    imgpoints = [] 
    
    for name in images:
        img = cv.imread(name)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        ret, corners = cv.findChessboardCorners(gray,(wc, hc),None)    
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            imgpoints.append(corners2)
        else:
            print(name, ret)
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return [mtx, dist]
    
os.makedirs("Output", exist_ok=True) # Result Images
Input_folder = getfolderlist("Input")
for f in Input_folder:
    Cam = Init_Undistortion()
    os.makedirs("Output/"+f, exist_ok=True)
    Input_list = getfilelist("Input/"+f)
    Cnt = 0
    HDRCnt = 0
    L = Input_list
    PNG_list = {}
    Base_list = L[0]
    Image_list = L[1]
    Silhouette = 0
    
    Base_handle = {}
    for v in list(Base_list.values()):
        Base_handle[v[2]] = OpenImage(v)
        if Base_handle[v[2]] == -1:
            raise Exception("[Base Image Error] {} is not a vaild image file".format(v[1]))
    print("Base : {:d} Images are Loaded.".format(len(L[0])))
    for k, v in enumerate(Image_list.values()):
        Image_handle = []
        PNG_list = []
        err_list = []
        for i, img in enumerate(v):
            itmp = OpenImage(img)
            if itmp == -1:
                print("[Image Error] {} is not a vaild image file".format(img[1]))
                err_list.append(i)
            else:
                Image_handle.append(itmp)
        for i in err_list:
            v.pop(i)
        for i in range(len(Image_handle)):
            Image_handle[i] = IntensityCorrection(Image_handle[i],Base_handle,v[i][2])
            PNG_list.append(AddPNGfile(v[i],Image_handle[i]))
            CloseImage(Image_handle[i])
            print("{} processed.".format(v[i][1]).replace(".dng",".png"))
            Cnt += 1
        if k == 0:
            Silhouette = SavetoHDR(PNG_list,f,Cam).astype('int32')
        else:
            Silhouette = np.add(Silhouette,SavetoHDR(PNG_list,f,Cam),dtype=np.int32)
        HDRCnt += 1
        print()
    for v in list(Base_handle.values()):
        CloseImage(v)
    Silhouette //= HDRCnt
    Silhouette = np.clip(Silhouette, 0, 255).astype('uint8')
    cv.imwrite("./Output/"+f+"/Silhouette.png", Silhouette)
    print()
    print("Silhouette Mask Image ({}) is Processed.".format("Silhouette.png"))
    print("Total {} Images are Processed.".format(Cnt))
    print("Total {} HDR Images are Processed.".format(HDRCnt))


Base : 4 Images are Loaded.
20241227_053948_AD000_17_15.0.png processed.
20241227_053948_AD000_17_150.0.png processed.
20241227_053948_AD000_17_50.0.png processed.
20241227_053948_AD000_17_500.0.png processed.
HDR Imaging (20241227_053948_AD000) : 4 Images are Loaded.
./Output/20241227_053948/000.png saved.

20241227_053948_AD001_16_15.0.png processed.
20241227_053948_AD001_16_150.0.png processed.
20241227_053948_AD001_16_50.0.png processed.
20241227_053948_AD001_16_500.0.png processed.
HDR Imaging (20241227_053948_AD001) : 4 Images are Loaded.
./Output/20241227_053948/001.png saved.

20241227_053948_AD002_15_15.0.png processed.
20241227_053948_AD002_15_150.0.png processed.
20241227_053948_AD002_15_50.0.png processed.
20241227_053948_AD002_15_500.0.png processed.
HDR Imaging (20241227_053948_AD002) : 4 Images are Loaded.
./Output/20241227_053948/002.png saved.

20241227_053948_AD003_14_15.0.png processed.
20241227_053948_AD003_14_150.0.png processed.
20241227_053948_AD003_14_50.0.png p